<a href="https://colab.research.google.com/github/aubricot/computer_vision_with_eol_images/blob/master/object_detection_for_image_tagging/plant_pollinator/plant_poll_generate_tags_yolov3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Using YOLO v3 pre-trained on Google Open Images to add plant-pollinator co-occurrence tags for ladybugs, beetles, and insects in plant images
---
*Last Updated 4 June 2021*   
Using a YOLOv3 model (downloaded from [here](https://github.com/AlexeyAB/darknet) ) pre-trained on [Google Open Images](https://storage.googleapis.com/openimages/web/visualizer/index.html?set=train&type=detection&c=%2Fm%2F03vt0) as a method to do customized, large-scale image processing. EOL Angiosperm images will be tagged for plant-pollinator co-occurrence using the detected insects. Tags will further extend EOLv3 image search functions.

## Installs & Imports
---

In [ ]:
# Mount google drive to import/export files
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
# For importing/exporting files, working with arrays, etc
import os
import pathlib
import six.moves.urllib as urllib
import sys
import tarfile
import zipfile
import numpy as np 
import csv
import matplotlib.pyplot as plt
import time
import pandas as pd

# For downloading images
!apt-get install aria2

# For drawing onto and plotting images
import matplotlib.pyplot as plt
from PIL import Image
from PIL import ImageColor
from PIL import ImageDraw
from PIL import ImageFont
from PIL import ImageOps
import cv2

%matplotlib inline
%config InlineBackend.figure_format = 'svg'

## Model preparation (only run once)
---

In [ ]:
# Install darknet

# TO DO: Type in the path to your working directory in form field to right
basewd = "/content/drive/MyDrive/train" #@param {type:"string"}
wd = 'darknet'
%cd $basewd

# Download darkflow (the tensorflow implementation of YOLO)
if os.path.exists("darknet"):
    %cd $wd

elif not os.path.exists("darknet"):
    !git clone https://github.com/AlexeyAB/darknet
    # Compile darkflow
    %cd $wd
    !python setup.py build_ext --inplace
    # Change makefile to have GPU and OPENCV enabled
    !sed -i 's/OPENCV=0/OPENCV=1/' Makefile
    !sed -i 's/GPU=0/GPU=1/' Makefile
    !sed -i 's/CUDNN=0/CUDNN=1/' Makefile
    !sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile
    # Download pretrained YOLOv3 weights for Open Images
    !wget https://pjreddie.com/media/files/yolov3-openimages.weights

# Verify CUDA version (for using GPU)
!/usr/local/cuda/bin/nvcc --version

# Make darknet
!make

/content/drive/My Drive/train/darknet2
/content/drive/My Drive/train/darknet2/darknet
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Wed_Jul_22_19:09:09_PDT_2020
Cuda compilation tools, release 11.0, V11.0.221
Build cuda_11.0_bu.TC445_37.28845127_0
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -DCUDNN_HALF -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -DCUDNN_HALF -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:926:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In fun

# Classify images
---

### Temporarily download images from EOL bundle to Google Drive (YOLO cannot directly parse URL images)

In [ ]:
# Download images
bundle = "https://editors.eol.org/other_files/bundle_images/files/images_for_Angiosperms_20K_breakdown_download_000031.txt" #@param {type:"string"}
df = pd.read_csv(bundle)

# Take subset of bundle
# TO DO: Change file name for each bundle/run abcd if doing 4 batches using dropdown form to right
ss = "human_present_tags_a" #@param ["plant_poll_coocc_tags_a", "plant_poll_coocc_tags_b", "plant_poll_coocc_tags_c", "plant_poll_coocc_tags_d"] {allow-input: true}
ss = ss + ".txt"

# Run in 4 batches of 5k images each (batch a is from 0-5000, b from 5000 to 10000, etc)
if "_a" in ss:
  a=0
  b=5000
elif "_b" in ss:
  a=5000
  b=10000
elif "_c" in ss:
  a=10000
  b=15000
elif "_d" in ss:
  a=15000
  b=20000

# Save subset to text file for image download
df = df.iloc[a:b]
outpath = "/content/drive/My Drive/train/darknet2/darknet/data/imgs/" + ss
df.to_csv(outpath, sep='\n', index=False, header=False)

# Download images (takes 7-10 min per 5k imgs, aria2 downloads 16imgs at a time)
%cd /content/drive/My Drive/train/darknet2/darknet/data/imgs
!aria2c -x 16 -s 1 -i $ss

# Check how many images downloaded
print("Number of images downloaded to Google Drive: ")
!ls . | wc -l

In [ ]:
# If images downloaded correctly, move text file to image_data/bundles
%cd ../
!mv imgs/*.txt img_info/

In [ ]:
# Make imgs.txt file to run images through YOLO for inference in batches
%cd /content/drive/My Drive/train/darknet2/darknet/data/

import glob
import os

path = "/content/drive/My Drive/train/darknet2/darknet/data/imgs"
inf_ss = path+'/'+ss
with open(inf_ss, 'w', encoding='utf-8') as f:
  for dir, dirs, files in os.walk(path):
    files = [fn for fn in files]
    for fn in files:
      if 'txt' not in fn:
        out = "data/imgs/" + fn
        f.writelines(out + '\n')

# Inspect imgs.txt file to confirm length and content
df = pd.read_csv(inf_ss, header=None)
df.head()
print(len(df))

### Run images through trained model
These steps take ~3 hours for 5,000 images

In [ ]:
# this creates a symbolic link so that now the path /content/gdrive/My\ Drive/ is equal to /mydrive
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

# TO DO: In next bloc, change inference image file list name at end of line after "<" to match inf_ss defined above
# ex: data/imgs/plant_poll_coocc_tags_a.txt
print("filename to copy-paste into code block below:", os.path.basename(inf_ss))

In [ ]:
# TO DO: Change inference image file list name at end of line after "<" to match inf_ss defined above
%cd /content/drive/My Drive/train/darknet2/darknet

# darknet run with external output flag to print bounding box coordinates
!./darknet detector test cfg/openimages.data cfg/yolov3-openimages.cfg yolov3-openimages.weights -dont_show -save_labels < data/imgs/plant_poll_coocc_tags_d.txt

#### To run individual images through by filename and display results

In [ ]:
# TO DO: Change inference image file list name at end of line after "<" to match inf_ss defined above
%cd /content/drive/My Drive/train/darknet2/darknet

# darknet run with external output flag to print bounding box coordinates
!./darknet detector test cfg/openimages.data cfg/yolov3-openimages.cfg yolov3-openimages.weights data/imgs/caterpillar_3.jpg

In [ ]:
# define helper functions
def imShow(path):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()

imShow('predictions.jpg')

### Post-process model output

In [ ]:
# Combine individual prediction files for each image to all_predictions.txt

# Delete image file list for inference
inf_ss = 'data/imgs/' + os.path.basename(inf_ss)
!rm $inf_ss

# Combine individual text files and image filenames into all_predictions.txt
fns = os.listdir('data/imgs')
with open('data/results/all_predictions.txt', 'w') as outfile:
  header = "class_id x y w h img_id"
  outfile.write(header + "\n")
  for fn in fns:
        if 'txt' in fn:
          with open('data/imgs/'+fn) as infile:
            lines = infile.readlines()
            newlines = [''.join([x.strip(), ' ' + os.path.splitext(fn)[0] + '\n']) for x in lines]
            outfile.writelines(newlines)

# Inspect saved predictions
df = pd.read_csv('data/results/all_predictions.txt')
print(df.head())

# Delete all individual prediction files
!rm -r data/imgs/*.txt

# Delete all image files now that they have been used for inference
!rm -r data/imgs/*

In [ ]:
# Create final predictions dataframe with class names (instead of numbers) and image urls
# EOL image url bundle
df = pd.read_csv(bundle)
df.columns = ['url']
print(df)

# Model predictions with number-coded classes
predict = pd.read_csv('data/results/all_predictions.txt', header=0, sep=" ")
predict.class_id = predict.class_id - 1 #class_id counts started from 1 instead of 0 from YOLO
print(predict)

# Add class names to model predictions
classnames = pd.read_table('data/openimages.names')
classnames.columns = ['classname']
#print(classnames)
tag_df = predict.copy()
di = pd.Series(classnames.classname.values,index=classnames.index).to_dict()
tag_df.replace({"class_id":di}, inplace=True)
tag_df['class_id'] = tag_df['class_id'].astype(str)
print(tag_df)

# Add urls to model predictions
map_urls = df.copy()
img_ids = map_urls['url'].apply(lambda x: os.path.splitext((os.path.basename(x)))[0])
map_urls['img_id'] = img_ids
#print(map_urls)

tag_df.set_index('img_id', inplace=True, drop=True)
map_urls.set_index('img_id', inplace=True, drop=True)
mapped_tagdf = tag_df.merge(map_urls, left_index=True, right_index=True)
mapped_tagdf.reset_index(drop=False, inplace=True)
mapped_tagdf.drop_duplicates(inplace=True, ignore_index=True)
print(mapped_tagdf.head())

# Save final tags to file
fn = os.path.splitext(os.path.basename(inf_ss))[0]
outpath = 'data/results/' + fn + '.tsv'
mapped_tagdf.to_csv(outpath, sep="\t", index=False)

# Combine tag files A-D
---

In [ ]:
# Write header row of output tagging file
# TO DO: Change file name for each bundle/run abcd if doing 4 batches using dropdown form to right
tags_file = "plant_poll_coocc_tags_d" #@param {type:"string"}
tags_fpath = "/content/drive/My Drive/train/darknet2/darknet/data/results/" + tags_file + ".tsv"

# Combine exported model predictions and confidence values from above to one dataframe
fpath =  os.path.splitext(tags_fpath)[0]
base = fpath.rsplit('_',1)[0] + '_'
exts = ['a.tsv', 'b.tsv', 'd.tsv']
#exts = ['a.tsv', 'b.tsv', 'c.tsv', 'd.tsv']
all_filenames = [base + e for e in exts]
df1 = pd.concat([pd.read_csv(f, sep='\t', header=0, na_filter = False) for f in all_filenames], ignore_index=True)

# Filter for desired classes
filter = ['Butterfly', 'Insect', 'Beetle', 'Ant', 'Bat (Animal)', 'Bird', 'Bee', \
          'Invertebrate', 'Animal']
pattern = '|'.join(filter)
df = df1.copy()
print(df.class_id)
df.loc[df['class_id'].str.contains(pattern), 'class_id'] = 'Pollinator'
print(df.class_id[df.class_id.str.contains(pattern)])
print(len(df.class_id[df.class_id.str.contains(pattern)]))
df.loc[~df.class_id.str.contains(pattern), 'class_id'] = 'None'
print(df[~df.class_id.str.contains(pattern)])

# Write results to tsv
print(df.head())
outfpath = base + 'finaltags.tsv'
df.to_csv(outfpath, sep='\t', index=False)

# Display predictions on images
---
Inspect detection results and verify that they are as expected

In [ ]:
# TO DO: Do you want to use the tagging file exported above?
use_outfpath = "no" #@param ["yes", "no"]
# If no, choose other path to use
otherpath = "data/results/plant_poll_coocc_tags_finaltags.tsv" #@param {type:"string"}
if use_outfpath == "yes":
  outfpath = outfpath
else:
  outfpath = otherpath
df = pd.read_csv(outfpath, sep="\t", header=0)
print(df.head())

# For uploading an image from url
# Modified from https://www.pyimagesearch.com/2015/03/02/convert-url-to-image-with-python-and-opencv/
def url_to_image(url):
  resp = urllib.request.urlopen(url)
  image = np.asarray(bytearray(resp.read()), dtype="uint8")
  image = cv2.imdecode(image, cv2.IMREAD_COLOR)
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
 
  return image

In [ ]:
# Display crop dimensions on images
# Adjust line below to see up to 50 images displayed at a time
a = 0 #@param {type:"slider", min:0, max:5000, step:50}
b = a+50

for i, row in df.iloc[a:b].iterrows():
  # Read in image 
  url = df['url'][i]
  img = url_to_image(url)
  h,w = img.shape[:2]
  # Define variables needed to draw bounding box on image
  xmin = round((df['x'][i] - (df['w'][i]/2))*w)
  if (xmin < 0): xmin = 0
  ymin = round((df['y'][i] - (df['h'][i]/2))*h)
  if (ymin < 0): ymin = 0
  xmax = round(xmin + (df['w'][i]) * w)
  if (xmax > w-1): xmax = w-1
  ymax = round(ymin + (df['h'][i].astype(int)) * h)
  if (ymax > 0): ymax = h-1

  # Set box/font color and size
  maxdim = max(df['w'][i],df['h'][i])
  fontScale = 1
  box_col = (255, 0, 157)
  
  # Set box/font color and size
  maxdim = max(df['w'][i],df['h'][i])
  fontScale = 1
  box_col = (255, 0, 157)

  # Draw tag label on image
  tag = df['class_id'][i]
  image_wbox = cv2.putText(img, tag, (xmin+7, ymax-12), cv2.FONT_HERSHEY_SIMPLEX, fontScale, box_col, 2, cv2.LINE_AA)  
  
  # Draw box label on image
  image_wbox = cv2.rectangle(img, (xmin, ymax), (xmax, ymin), box_col, 5)
  
  # Plot and show cropping boxes on images
  _, ax = plt.subplots(figsize=(10, 10))
  ax.imshow(image_wbox)
  # Display image URL above image to facilitate troubleshooting/fine-tuning of data reformatting and tidying steps in convert_bboxdims.py or preprocessing.ipynb
  plt.title('{}'.format(url, xmin, ymin, xmax, ymax))